In [2]:
import pandas as pd
import os

In [3]:
directory = ['balance-sheet', 'cash-flow-statement', 'financial-statements', 'financial-ratios']

In [4]:
ticker_to_df = {}
for d in directory:
    # iterate through files in directory
    for b in os.listdir(d):
        for filename in os.listdir(d+ '/' +b):
            # Read csv file
            df = pd.read_csv(d+ '/' + b + '/' + filename)
            ticker = filename.split("-")[0]
            # Add suffix to column names of dataframe

            df.columns = ['Date'] + [str(col) + "-" + b for col in df.columns[1:]]
            # Add dataframe to dictionary
            if ticker not in ticker_to_df:
                ticker_to_df[ticker+"-"+b] = df
            else:
                ticker_to_df[ticker+"-"+b] = pd.merge(ticker_to_df[ticker], df, on='Date', how='outer')
            


In [16]:
# Save df as csv file
for ticker in ticker_to_df:
    ticker_to_df[ticker].to_csv('combined/' + ticker + '.csv', index=False)

In [21]:
# Concatenate df row wise
annual_df = 0
quarterly_df = 0
for ticker in ticker_to_df:
    if ticker.split('-')[1] == 'annual':
        if isinstance(annual_df, int):
            annual_df = ticker_to_df[ticker]
            continue
        annual_df = pd.concat((annual_df, ticker_to_df[ticker]), axis=0, ignore_index=True)
    else:
        if isinstance(quarterly_df, int):
            quarterly_df = ticker_to_df[ticker]
            continue
        quarterly_df = pd.concat((quarterly_df, ticker_to_df[ticker]), axis=0, ignore_index=True)

1


In [25]:
# Get column-wise min and max values
annual_min = annual_df.min()
annual_max = annual_df.max()
quarterly_min = quarterly_df.min()
quarterly_max = quarterly_df.max()
print(annual_min, annual_max, quarterly_min, quarterly_max)

Date                                    2009-01-31
current-ratio-annual                         0.136
long-term-debt-capital-annual             -16.3863
debt-equity-ratio-annual                  -776.587
gross-margin-annual                      -471.3438
operating-margin-annual                   -15071.1
ebit-margin-annual                        -15071.1
ebitda-margin-annual                     -14814.13
pre-tax-profit-margin-annual             -18097.79
net-profit-margin-annual                 -17746.38
asset-turnover-annual                       0.0028
inventory-turnover-annual                   0.1702
receiveable-turnover-annual                 0.0566
days-sales-in-receivables-annual            0.1226
roe-annual                               -10591.31
return-on-tangible-equity-annual         -76191.98
roa-annual                               -210.9951
roi-annual                               -1997.717
book-value-per-share-annual              -21952.85
operating-cash-flow-per-share-a

In [27]:
for ticker in ticker_to_df:
    if ticker.split('-')[1] == 'annual':
        # Normalize df except for string columns
        for col in ticker_to_df[ticker].columns:
            if col != 'Date':
                ticker_to_df[ticker][col] = ticker_to_df[ticker][col] / annual_max[col]
    else:
        # Normalize df
        for col in ticker_to_df[ticker].columns:
            if col != 'Date':
                ticker_to_df[ticker][col] = ticker_to_df[ticker][col] / quarterly_max[col]

    print(ticker_to_df[ticker].min(), ticker_to_df[ticker].max())

Date                                    2009-12-31
current-ratio-annual                       0.05165
long-term-debt-capital-annual             0.009373
debt-equity-ratio-annual                  0.001279
gross-margin-annual                       0.280269
operating-margin-annual                   0.114837
ebit-margin-annual                        0.114837
ebitda-margin-annual                      0.156993
pre-tax-profit-margin-annual              0.043293
net-profit-margin-annual                  0.028513
asset-turnover-annual                     0.110823
inventory-turnover-annual                 0.021252
receiveable-turnover-annual               0.001557
days-sales-in-receivables-annual          0.006924
roe-annual                                0.000583
return-on-tangible-equity-annual         -0.006486
roa-annual                                 0.04029
roi-annual                                0.003347
book-value-per-share-annual               0.000295
operating-cash-flow-per-share-a

In [28]:
# Save df as csv file
for ticker in ticker_to_df:
    ticker_to_df[ticker].to_csv('combined-preprocessed/' + ticker + '.csv', index=False)